In [1]:
# Import packages
import os
import random
from PIL import Image
from torch import nn
import torch 
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import json 
from src.config import PROCESSED_DATA_DIR, RAW_DATA_DIR
import torch.optim as optim

from loguru import logger

from utils import train_validate_model, modify_model_output, test_model

import time
import wandb

image_path = PROCESSED_DATA_DIR / "pizza_hamburger_hotdog_20_percent"
train_dir = image_path / 'train' 
test_dir = image_path / 'test'
valid_dir = image_path / 'valid'

device ='cuda' if torch.cuda.is_available() else 'cpu'

2024-06-23 13:01:05.063 | INFO     | src.config:<module>:11 - PROJ_ROOT path is: C:\Git\hamburger-hotdog-pizza-classifier


In [2]:
wandb.init(project='pizza_hamburger_hotdog_20_percent')

wandb: Currently logged in as: dtiourine. Use `wandb login --relogin` to force relogin


In [3]:
config_path = 'params.json'

with open(config_path, 'r') as config_file:
   config = json.load(config_file)
#print(json.dumps(config, indent=2))

In [4]:
# Load hyperparams from params.json

learning_rate = config['model_params']['learning_rate']
batch_size = config['model_params']['batch_size']
num_epochs = config['model_params']['num_epochs']
dropout_rate = config['model_params']['dropout_rate']
optimizer = config['model_params']['optimizer']
loss_function = config['model_params']['loss_function']
metrics = config['model_params']['metrics']
output_shape = config['model_params']['output_shape']

In [5]:
# Prepare data into dataloader
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
        transforms.ToTensor()
])

train_data = ImageFolder(train_dir, transform=transform)
valid_data = ImageFolder(valid_dir, transform=transform)
test_data = ImageFolder(test_dir, transform=transform)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

# Start with Pre-Trained Models

Let's choose some popular CNN architectures as a starting point

First, we need to load these pretrained models

In [6]:
num_classes = 3
alexnet = modify_model_output('alexnet', num_classes, device)
vgg16 = modify_model_output('vgg16', num_classes, device)
resnet50 = modify_model_output('resnet50', num_classes, device)

## 1. Trying ResNet50

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet50.parameters(), lr=0.001, momentum=0.3)
model_name = "ResNet50"
date_time = time.time()
logger.add(f"logs/{model_name}/training_log-{date_time}.log", format="{time} {level} {message}", level="INFO")
train_validate_model(num_epochs=10, model=resnet50, train_loader=train_loader, valid_loader=valid_loader, criterion=criterion, optimizer=optimizer, device=device)

Training Epoch 1/10: 100%|██████████| 5/5 [00:01<00:00,  3.55it/s]


2024-06-23 13:01:10.038 | INFO     | utils:train_validate_model:88 - Epoch 1, Train Loss: 1.0873, Train Accuracy: 39.26%


Validation Epoch 1/10: 100%|██████████| 3/3 [00:00<00:00,  6.09it/s]


2024-06-23 13:01:10.531 | INFO     | utils:train_validate_model:111 - Epoch 1, Validation Loss: 1.1012, Validation Accuracy: 30.56%


Training Epoch 2/10: 100%|██████████| 5/5 [00:01<00:00,  3.33it/s]


...

2024-06-23 13:01:23.576 | INFO     | utils:train_validate_model:111 - Epoch 9, Validation Loss: 1.0385, Validation Accuracy: 61.67%


Training Epoch 10/10: 100%|██████████| 5/5 [00:01<00:00,  4.74it/s]


2024-06-23 13:01:24.632 | INFO     | utils:train_validate_model:88 - Epoch 10, Train Loss: 1.0305, Train Accuracy: 67.04%


Validation Epoch 10/10: 100%|██████████| 3/3 [00:00<00:00,  5.03it/s]

2024-06-23 13:01:25.228 | INFO     | utils:train_validate_model:111 - Epoch 10, Validation Loss: 1.0346, Validation Accuracy: 61.67%


## 2. Trying VGG16

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(vgg16.parameters(), lr=0.001, momentum=0.3)
model_name = "VGG16"
date_time = time.time()
logger.add(f"logs/{model_name}/training_log-{date_time}.log", format="{time} {level} {message}", level="INFO")
train_validate_model(num_epochs=20, model=vgg16, train_loader=train_loader, valid_loader=valid_loader, criterion=criterion, optimizer=optimizer, device=device)

Training Epoch 1/20: 100%|██████████| 5/5 [00:01<00:00,  3.25it/s]


2024-06-23 13:06:02.474 | INFO     | utils:train_validate_model:88 - Epoch 1, Train Loss: 0.1602, Train Accuracy: 93.33%


Validation Epoch 1/20: 100%|██████████| 3/3 [00:00<00:00,  5.21it/s]


2024-06-23 13:06:03.052 | INFO     | utils:train_validate_model:111 - Epoch 1, Validation Loss: 0.2882, Validation Accuracy: 88.89%


Training Epoch 2/20: 100%|██████████| 5/5 [00:02<00:00,  2.30it/s]


...

2024-06-23 13:06:50.836 | INFO     | utils:train_validate_model:111 - Epoch 19, Validation Loss: 0.2683, Validation Accuracy: 89.44%


Training Epoch 20/20: 100%|██████████| 5/5 [00:01<00:00,  3.78it/s]


2024-06-23 13:06:52.160 | INFO     | utils:train_validate_model:88 - Epoch 20, Train Loss: 0.0628, Train Accuracy: 98.15%


Validation Epoch 20/20: 100%|██████████| 3/3 [00:01<00:00,  2.12it/s]

2024-06-23 13:06:53.578 | INFO     | utils:train_validate_model:111 - Epoch 20, Validation Loss: 0.2942, Validation Accuracy: 89.44%


## 3. Trying AlexNet

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(alexnet.parameters(), lr=0.001, momentum=0.3)
model_name = "AlexNet"
date_time = time.time()
logger.add(f"logs/{model_name}/training_log-{date_time}.log", format="{time} {level} {message}", level="INFO")
train_validate_model(num_epochs=10, model=alexnet, train_loader=train_loader, valid_loader=valid_loader, criterion=criterion, optimizer=optimizer, device=device)

Training Epoch 1/10: 100%|██████████| 5/5 [00:00<00:00,  5.13it/s]


2024-06-23 13:07:23.732 | INFO     | utils:train_validate_model:88 - Epoch 1, Train Loss: 1.1335, Train Accuracy: 42.59%


Validation Epoch 1/10: 100%|██████████| 3/3 [00:00<00:00,  6.62it/s]


2024-06-23 13:07:24.190 | INFO     | utils:train_validate_model:111 - Epoch 1, Validation Loss: 0.9382, Validation Accuracy: 55.56%


Training Epoch 2/10: 100%|██████████| 5/5 [00:00<00:00,  5.16it/s]


...

2024-06-23 13:07:36.562 | INFO     | utils:train_validate_model:111 - Epoch 9, Validation Loss: 0.5620, Validation Accuracy: 76.67%


Training Epoch 10/10: 100%|██████████| 5/5 [00:01<00:00,  4.41it/s]


2024-06-23 13:07:37.705 | INFO     | utils:train_validate_model:88 - Epoch 10, Train Loss: 0.4082, Train Accuracy: 82.96%


Validation Epoch 10/10: 100%|██████████| 3/3 [00:00<00:00,  4.80it/s]

2024-06-23 13:07:38.333 | INFO     | utils:train_validate_model:111 - Epoch 10, Validation Loss: 0.5750, Validation Accuracy: 76.11%
